<h1> Create Response To Customer Reviews - Sentiment Analysis </h1>
<br>

Before starting, please make sure this notebook is using **conda_python3** kernel from the top right!

Run all the cells and inspect the output of each cell.

### Introduction

In this notebook, you will create response to customer reviews using Text-to-Text LLM from Amazon Bedrock. We will create the prompt that instructs the LLM to add a sentence to the response if the customer review is negative. 

![Sentiment Analysis](../images/sentiment-analysis.png)

### Install required dependencies

**Important:** You may see an error or a warning that "you may need to restart the kernel" from the following cell. **Ignore** and proceed with the next cells. 

In [ ]:
%pip install --quiet --no-build-isolation --upgrade \
    "boto3==1.28.63" \
    "awscli==1.29.63" \
    "botocore==1.31.63" \
    "langchain==0.0.309"

<h3> Import required packages </h3>

In [ ]:
import json
import os
import sys
import boto3
import botocore
from langchain import PromptTemplate
from langchain.llms.bedrock import Bedrock

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

<h3> Initialize Bedrock client </h3><br>

We will use this client to invoke the Bedrock API and call the LLM. 

In [ ]:
 boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

<h3> Initialize the LLM </h3>
<p> Using Langchain, initialize the ClaudeInstant LLM. </p>

In [ ]:
# Passing Anthropic Claude as model ID

textgen_llm = Bedrock(
            model_id="anthropic.claude-instant-v1",
            client=boto3_bedrock
        )

### Prompt template 

Create a prompt template with the variables: product name, customer name, manager name, manager contact, customer review, response length. 

Using this prompt, we will ask the LLM to add a sentence to reach out to the manager directly with their phone number **IF the customer review is negative**.  

Read the template parameter carefully to understand the prompting instructions.

In [ ]:
prompt_template = PromptTemplate(
            input_variables=["product_name","customer_name","manager_name","email","phone","length","review"], 
            template="""
                    Human: 
                    
                    I'm the manager of re:Invent retails. 
                    
                    Draft a response for the review of the product {product_name} from our customer {customer_name}. 
                    The number of words should be less than {length}. 
                    
                    My contact information is email: {email}, phone: {phone}.
                    
                    <customer_review>
                        {review}
                    <customer_review>

                    <example_response_pattern>
                    
                        Dear <customer_name>,
                        <content_body>

                        <if negative review> 
                            Don't hesitate to reach out to me at {phone}.
                        <end if> 

                        Sincerely,
                        {manager_name}
                        <signature>
                        {email}
                        
                    </example_response_pattern>
                    
                    Assistant:
                    
                    """
                )

Do you notice this line? 

\<if negative review\> <br>
  &nbsp;&nbsp;&nbsp; &nbsp;    Don't hesitate to reach out to me at {phone}. <br>
\<end if\> <br>

Later in this notebook, you will see how this works. 

<h3>Create sample input variables</h3>

We create input variables that will be used to construct our prompt which will be passed to the LLM to create review response.

In [ ]:
# Product that was reviewed by the customer 
product_name="Treadlite Shoe"

# Customer name
customer_name = "David Bush"

# Manager name
manager_name = "Austin Lee"

# Store manager contact
my_email = "retail.manager@reinventretails.com"
my_phone = "024-555-1234"

# Review posted
customer_review = """I've owned these shoes for about 6 months now and have put over 300 miles on them. 
                 They are super lightweight and provide excellent cushioning and support for long runs. 
                 The breathable mesh keeps my feet from overheating even on hot summer days. 
                 The only minor issue is that the tread is starting to show some light wear 
                 but for a $200 shoe I'm really impressed with the durability. 
                 Overall these are a fantastic value for any serious runner."""

# Length of the desired product description (generated from LLM)
max_length = 100

<p> Now, let's construct the prompt by passing all the input variables to the prompt template. </p>

In [ ]:
prompt = prompt_template.format(product_name=product_name,
                                customer_name=customer_name,
                                manager_name=manager_name,
                                email=my_email,
                                phone=my_phone,
                                length=max_length,
                                review=customer_review)

<h3> Call Bedrock with the constructed prompt </h3>

In [ ]:
response = textgen_llm(prompt)
print_ww(response)

<h4> Now let's pass in a negative review to Bedrock to see how the model responds </h4>

In [ ]:
customer_name="Anthony Lopez"

# Negative review posted
customer_review = """Treadlite shoes promise lightweight comfort, but they failed to deliver for me. 
                     Within a few weeks of regular use, the thin material started wearing down already. 
                     There are holes forming on the sides and small tears along the seams. 
                     The fabric just does not feel durable enough. 
                     Additionally, I found these shoes to be poorly cushioned. 
                     After a 5 mile run, my feet and knees were sore from the lack of support and bounce. 
                     It was like running directly on the hard pavement. 
                     The shoe provides almost no impact absorption for a runner.
                     The sizing is also off. I bought my normal size but the shoes feel restrictive, 
                     like my feet are being squeezed. They did not stretch or mold to the 
                     shape of my foot over time like other running shoes. 
                     It's an uncomfortable fit that leaves my feet feeling constricted after runs.
                     Between the lack of cushioning, poor durability of materials, and sizing issues, 
                     these Treadlite shoes have been a big disappointment. 
                     For a brand focused on running, the design flaws mean they are not well-suited 
                     for the needs of active individuals. I cannot recommend these shoes 
                     and would not purchase from this brand again based on my experience. 
                     Runners deserve better quality and performance than what Treadlite provided."""

In [ ]:
prompt = prompt_template.format(product_name=product_name,
                                customer_name=customer_name,
                                manager_name=manager_name,
                                email=my_email,
                                phone=my_phone,
                                length=max_length,
                                review=customer_review)

In [ ]:
response = textgen_llm(prompt)
print_ww(response)

Notice the extra sentence added by the model in the message body providing store manager's phone number and offering a call. It is because of our prompting instruction for negative reviews within \<example_response_pattern\>\<\/example_response_pattern\>. 
                    
\<example_response_pattern\> \
<br>
    &nbsp;&nbsp;&nbsp;&nbsp;Dear \<customer_name\>, \
    &nbsp;&nbsp;&nbsp;&nbsp;\<content_body\> \
    <br>
    <b>
    &nbsp;&nbsp;&nbsp;&nbsp;\<if negative review\> \
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Don't hesitate to reach out to me at {phone}. \
    &nbsp;&nbsp;&nbsp;&nbsp;\<end if\> \
    </b>
    <br>
    &nbsp;&nbsp;&nbsp;&nbsp;Sincerely, \
    &nbsp;&nbsp;&nbsp;&nbsp;\<signature\> \
    &nbsp;&nbsp;&nbsp;&nbsp;{email} \
    &nbsp;&nbsp;&nbsp;&nbsp;<br>
\<\/example_response_pattern\>

<h3> You've successfully drafted response to the customer reviews for a product with Amazon Bedrock!</h3>

Please stop the notebook kernel by selecting **Kernel -> Interrupt**.

#### Now, let's integrate this feature into our retail web application. Please go back to Workshop Studio and follow the instructions to build this feature using your Cloud9 IDE.